### Delete wrongly mapped recipes in planned meal table
1. get the recipes mapped to each meal plan
2. filter wrongly mapped recipes 
3. get the schedule id for worngly mapped
4. delete the wrongly mapped recipes using the API 

In [ ]:
import pandas as pd
import json
import pickle
import requests
import pathlib
import time
import numpy as np
import os
import requests
from itertools import chain
import ast

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def load_json(path):
    with open(path , 'r') as f:
        return json.load(f)

def export_json(data, path='result.json'):
    with open(path, 'w') as f:
        json.dump(data, f)

def load_pickle(path):
    with open(path , 'rb') as f:
        return pickle.load(f)
    
def export_pickle(data, path='result.pikle'):
    with open(path, 'wb') as f:
        pickle.dump(data, f)

#### get wrongly mapped recipes for meal plan 

In [ ]:
def get_plan_meals(plan_id):
    url = f"http://18.223.178.105:5000/api/v1/plan_management/plan/{plan_id}"
    response = requests.request("GET", url, headers={}, data="")
    return json.loads(response.text)

In [ ]:
def get_meal_map_details(plan_id):
    res = get_plan_meals(plan_id)
    temp = []
    for day, vals in res['plan_details'].items():
        for tm, rep in vals.items():
            rep = rep['recipes']
            for obj in rep:
                obj['day_name'] = day
                obj['time'] = tm
                temp.append(obj)
    temp = pd.DataFrame.from_dict(temp)
    temp.recipe_name = temp.recipe_name.str.lower()
    temp = temp.groupby(['day_name', 'time'])['recipe_name'].unique()
    temp = temp.reset_index()
    return temp

#### get the status csv containing mapping status like pending recipes to map, wrongly mapped recipes etc  

In [ ]:
def get_meal_mapping_status(df_diet_plan):
    df_result_list = []
    for plan_id, df_diet in df_diet_plan.groupby('plan_id'):
        try:
            diet_db = get_meal_map_details(plan_id)
            
            df_diet.Name = df_diet.Name.str.lower()
            df_diet = df_diet.groupby(['day_name', 'time'])['Name'].unique()
            df_diet = df_diet.reset_index()
            result= pd.merge(df_diet, diet_db, left_on=['day_name', 'time'], right_on=['day_name', 'time'], how='left')
            result.recipe_name = result.recipe_name.fillna("").apply(list)
            result['pending_recipe'] = result.apply(lambda row : list(set(row['Name'])-set(row['recipe_name'])), axis=1)
            result['mismatch_recipe'] = result.apply(lambda row : list(set(row['recipe_name'])-set(row['Name'])), axis=1)
            result['recipe_per_schedule'] = result.apply(lambda row : len(set(row['Name'])), axis=1)
            result['mapped_recipe'] = result.apply(lambda row : len(set(row['Name'])), axis=1)
            result['pending_recipe_per_schedule'] = result.apply(lambda row : len(set(row['pending_recipe'])), axis=1)
            result['wrong_count'] = result.apply(lambda row : len(set(row['mismatch_recipe'])), axis=1)
            result['plan_id'] = plan_id
            df_result_list.append(result)
            print('Processed ', plan_id)
        except Exception as e:
            print('\t Exception ', e)
            print('\t Failed to process ', plan_id)

    df_result = pd.concat(df_result_list, ignore_index=True)
    df_result.to_csv('meal_mapping_status.csv', index=False)
    return df_result

In [ ]:
def delete_planned_meal_api(payload):
    url = "http://18.223.178.105:5000/api/v1/plan_management/delete-planned-meal"
    headers = { "Content-Type" : "application/json;charset=utf-8"}
    payload = json.dumps(payload)
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)

In [ ]:
def delete_recipe_mapping(schedule_id, recipes):
    if recipes:
        for recipe_name in recipes:
            payload = {"recipe_id":recipe_name_id.get(recipe_name),
                    "schedule_id": schedule_id}
            print(payload)
            delete_planned_meal_api(payload)
            

In [ ]:
def get_wrong_recipe_maps(df_diet_plan, df_meal_status):
    x = df_meal_status[['day_name', 'time', 'plan_id', 'mismatch_recipe']]
    y = df_diet_plan[['time', 'day_name', 'plan_id', 'plan_schedule_id']]
    y = y.drop_duplicates()
    
    return pd.merge(x,y, on=['time', 'day_name', 'plan_id'], how='left')

#### get wrongly mapped recipe list 

In [ ]:
df_diet_plan = pd.read_csv('/home/syed/Downloads/Meal_plan_mapping_format_28_diet_Allplans.csv')
df_meal_status = get_meal_mapping_status(df_diet_plan)
# df_meal_status = pd.read_csv('./meal_mapping_status.csv')
df_delete = get_wrong_recipe_maps(df_diet_plan, df_meal_status)

#### delete wrongly mapped recipes 

In [ ]:
df_delete.apply(lambda row: delete_recipe_mapping(row['plan_schedule_id'], row['mismatch_recipe']), axis=1)